In [32]:
import time
import torch
import torch.nn as nn
from einops import einsum, rearrange

iter_num = 20
device = 'mps'
if device == 'mps':
    func = torch.mps.synchronize
else:
    func = torch.cpu.synchronize


num = 10
N1_ = 2 ** num
N2_ = 1

# TensorCore是不是比较喜欢做那种, 扁矩阵的乘法(似乎是的)

In [35]:
for i in range(num + 1):
    N2 = N1_ // (2 ** i)
    N1 = N2_ * (2 ** i)
    print(f"N1: {N1}, N2: {N2}")
    R1 = torch.randn(N1, N1, device=device, dtype=torch.float32)
    R2 = torch.randn(N2, N2, device=device, dtype=torch.float32)
    # 权重的形状是(C_out, C_in)
    weight = torch.randn(N1 * N2, N1 * N2, device=device, dtype=torch.float32)
    
    # # 在沿着输入维度旋转的操作
    with torch.inference_mode():
        Q = torch.kron(R1, R2)
        # print(20 * '==')
        for _ in range(3):
            start = time.time()
            for _ in range(iter_num):
                output_baseline = weight @ Q
            func()
        print(f"time: {time.time() - start}")
        
        # print(20 * '==')
        for _ in range(3):
            start = time.time()
            for _ in range(iter_num):
                output2 = torch.einsum('ij,aik,km->ajm', R1, rearrange(weight, 'b (h c)->b h c', h=N1), R2).reshape(N1 * N2, -1)
            func()
        print(f"time: {time.time() - start}")
        
        # print(20 * '==')
        for _ in range(3):
            start = time.time()
            for _ in range(iter_num):
                output3 = einsum(R1, rearrange(weight, 'b (h c)->b h c', h=N1), R2, 'i j, a i k, k m -> a j m').reshape(N1 * N2, -1)
            func()
        print(f"time: {time.time() - start}")
        
        # print(output_baseline.shape, output2.shape, output3.shape)
        print((output_baseline - output2).abs().max().detach().cpu().item())
        # # 目测爱因斯坦
        # print((output3 - output2).abs().max())
        # print(output_baseline.sum(), output2.sum(), output3.sum())

N1: 1, N2: 1024
time: 0.014331340789794922
time: 0.07174801826477051
time: 0.07397580146789551
3.0517578125e-05
N1: 2, N2: 512
time: 0.014024019241333008
time: 0.02248215675354004
time: 0.022209882736206055
0.0001068115234375
N1: 4, N2: 256
time: 0.01253819465637207
time: 0.02038407325744629
time: 0.020218849182128906
0.0003204345703125
N1: 8, N2: 128
time: 0.012588024139404297
time: 0.041749000549316406
time: 0.04278898239135742
0.0003204345703125
N1: 16, N2: 64
time: 0.012185811996459961
time: 0.0270688533782959
time: 0.019181251525878906
0.00022125244140625
N1: 32, N2: 32
time: 0.01222681999206543
time: 0.015288114547729492
time: 0.016564130783081055
0.0002593994140625
N1: 64, N2: 16
time: 0.012888908386230469
time: 0.020623207092285156
time: 0.02318406105041504
0.0002593994140625
N1: 128, N2: 8
time: 0.013580083847045898
time: 0.05196881294250488
time: 0.05097794532775879
0.00030517578125
N1: 256, N2: 4
time: 0.012922048568725586
time: 0.05341005325317383
time: 0.05140519142150879
